<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Carga_Archivos_Barra_Progresos_Estados_Alarmas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Carga de datos con barra de progreso - Version 1 - Funciona OK

In [1]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar datos
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Detectar tipo de instrumento según nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# Widgets de carga y salida
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)
output = widgets.Output()
barra_progreso = widgets.IntProgress(value=0, min=0, max=1, description='Progreso:', bar_style='info')

# Función principal de carga con barra de progreso
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        total = len(archivos)
        barra_progreso.max = total
        barra_progreso.value = 0
        barra_progreso.bar_style = 'info'
        display(barra_progreso)

        for i, (nombre_archivo, archivo_info) in enumerate(archivos.items(), start=1):
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                print(f"⏳ Cargando: {nombre_archivo} ({i}/{total})")

                if not instrumento:
                    print(f"❌ Instrumento no reconocido: {nombre_archivo}")
                    barra_progreso.value = i
                    continue

                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")

            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

            barra_progreso.value = i

        barra_progreso.bar_style = 'success'
        print("✅ Carga finalizada.")
        mostrar_menu()

# Menú de visualización de DataFrames
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)


FileUpload(value={}, accept='.csv,.xlsx', description='Subir archivos', multiple=True, style=ButtonStyle(butto…

Output()

#Carga de datos con barra de progresos y avance porcentual - Version 2 - Agrega avance porcentaul - Funcion OK

In [ ]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar datos
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Detectar tipo de instrumento según nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# Widgets de carga y salida
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)
output = widgets.Output()
barra_progreso = widgets.FloatProgress(value=0, min=0, max=100, description='Progreso:', bar_style='info')
etiqueta_progreso = widgets.Label(value="0% completado")

# Función principal de carga con barra de progreso con porcentaje
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        total = len(archivos)
        barra_progreso.value = 0
        barra_progreso.bar_style = 'info'
        etiqueta_progreso.value = "0% completado"
        display(barra_progreso, etiqueta_progreso)

        for i, (nombre_archivo, archivo_info) in enumerate(archivos.items(), start=1):
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                print(f"⏳ Cargando: {nombre_archivo} ({i}/{total})")

                if not instrumento:
                    print(f"❌ Instrumento no reconocido: {nombre_archivo}")
                    porcentaje = (i / total) * 100
                    barra_progreso.value = porcentaje
                    etiqueta_progreso.value = f"{porcentaje:.0f}% completado"
                    continue

                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")

            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

            porcentaje = (i / total) * 100
            barra_progreso.value = porcentaje
            etiqueta_progreso.value = f"{porcentaje:.0f}% completado"

        barra_progreso.bar_style = 'success'
        etiqueta_progreso.value = "✅ Carga completada al 100%"
        print("✅ Carga finalizada.")
        mostrar_menu()

# Menú de visualización de DataFrames
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)


FileUpload(value={'AS175_20250608.csv': {'metadata': {'name': 'AS175_20250608.csv', 'type': 'text/csv', 'size'…

Output()

📊 Mostrando: EXTENSOMETRO (CSV)


,FECHA,EXTENSOMETRO,COTA_EXCAV._(MSNM),PROFUNDIDAD,Z/COTA_RELEV.,DIFERENCIAS_(MM),ACUMULADO_(MM),COTA_FINAL_EXCAV._(MSNM)
0,03/04/2022,EX-CH-1a,120.0,0.0,120.0,0.0,0.0,120.0
1,12/05/2022,EX-CH-1a,120.0,0.0,120.0,0.0,0.0,120.0
2,30/05/2022,EX-CH-1a,120.0,0.0,120.0,0.0,0.0,120.0
3,11/06/2022,EX-CH-1a,120.0,0.0,120.0,0.0,0.0,120.0
4,04/07/2022,EX-CH-1a,120.0,0.0,120.0,0.0,0.0,120.0


# Version Mejorada de Carga de Datos con Selectores y botones - Version 3 - Funciona OK

In [ ]:
import pandas as pd
import io
import base64
from datetime import datetime
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

# --- Configuración inicial ---
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return "puntos_fijos_mi"  # Valor por defecto si no se especifica
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# Crear widgets globales
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)
output = widgets.Output()
output_accion = widgets.Output()

# Los selectores y botones se crean una sola vez, fuera del menú y nunca dentro de un contexto de output.
instrumento_selector = widgets.Dropdown(
    options=instrumentos,
    description='Instrumento:',
    layout=widgets.Layout(width='300px')
)
origen_selector = widgets.Dropdown(
    options=['csv', 'xlsx'],
    description='Origen:',
    layout=widgets.Layout(width='200px')
)
boton_ver = widgets.Button(
    description='👁️ Ver Datos',
    button_style='success',
    layout=widgets.Layout(width='150px')
)
boton_descargar = widgets.Button(
    description='💾 Descargar',
    button_style='info',
    layout=widgets.Layout(width='150px')
)

def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        barra_progreso = widgets.FloatProgress(
            value=0, min=0, max=100, description='Progreso:', bar_style='info',
            layout=widgets.Layout(width='50%')
        )
        etiqueta_progreso = widgets.Label(value="0% completado")
        display(barra_progreso, etiqueta_progreso)

        total = len(archivos)
        archivos_exitosos = 0

        for i, (nombre_archivo, archivo_info) in enumerate(archivos.items(), start=1):
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                print(f"📄 Procesando: {nombre_archivo}")

                if not instrumento:
                    print(f"⚠️ No se pudo detectar el instrumento para: {nombre_archivo}")
                    porcentaje = (i / total) * 100
                    barra_progreso.value = porcentaje
                    etiqueta_progreso.value = f"{porcentaje:.0f}% completado"
                    continue

                # Procesar archivo según extensión
                if extension == 'csv':
                    try:
                        try:
                            df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                        except UnicodeDecodeError:
                            df = pd.read_csv(io.BytesIO(contenido), encoding='latin-1')

                        if not df.empty:
                            datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                            print(f"✅ CSV cargado para {instrumento}: {len(df)} filas")
                            archivos_exitosos += 1
                        else:
                            print(f"⚠️ Archivo CSV vacío: {nombre_archivo}")
                    except Exception as e:
                        print(f"❌ Error al leer CSV {nombre_archivo}: {str(e)}")

                elif extension == 'xlsx':
                    try:
                        df = pd.read_excel(io.BytesIO(contenido))
                        if not df.empty:
                            datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                            print(f"✅ XLSX cargado para {instrumento}: {len(df)} filas")
                            archivos_exitosos += 1
                        else:
                            print(f"⚠️ Archivo XLSX vacío: {nombre_archivo}")
                    except Exception as e:
                        print(f"❌ Error al leer XLSX {nombre_archivo}: {str(e)}")
                else:
                    print(f"⚠️ Extensión no soportada: {extension}")

            except Exception as e:
                print(f"❌ Error general al procesar {nombre_archivo}: {str(e)}")

            porcentaje = (i / total) * 100
            barra_progreso.value = porcentaje
            etiqueta_progreso.value = f"{porcentaje:.0f}% completado"

        barra_progreso.bar_style = 'success'
        etiqueta_progreso.value = f"✅ Carga completada: {archivos_exitosos}/{total} archivos procesados exitosamente"

        print(f"\n📊 Resumen de datos cargados:")
        for inst in instrumentos:
            csv_count = len(datos_csv[inst])
            xlsx_count = len(datos_xlsx[inst])
            if csv_count > 0 or xlsx_count > 0:
                print(f"  • {inst}: {csv_count} filas CSV, {xlsx_count} filas XLSX")

def ver_datos(b):
    with output_accion:
        clear_output(wait=True)
        instrumento = instrumento_selector.value
        origen = origen_selector.value
        df = datos_csv[instrumento] if origen == "csv" else datos_xlsx[instrumento]
        print(f"📋 Mostrando datos de: {instrumento} ({origen})")
        print("=" * 50)
        if df.empty:
            print("⚠️ No hay datos disponibles para este instrumento y origen.")
            print("💡 Asegúrate de que el archivo se haya cargado correctamente.")
        else:
            print(f"📊 Total de filas: {len(df)}")
            print(f"📊 Total de columnas: {len(df.columns)}")
            print(f"📊 Columnas: {list(df.columns)}")
            print("\n🔍 Primeras 20 filas:")
            print("-" * 50)
            display(df.head(20))

def descargar_datos(b):
    with output_accion:
        clear_output(wait=True)
        instrumento = instrumento_selector.value
        origen = origen_selector.value
        df = datos_csv[instrumento] if origen == "csv" else datos_xlsx[instrumento]
        print(f"💾 Preparando descarga: {instrumento} ({origen})")
        print("=" * 50)
        if df.empty:
            print("⚠️ No hay datos disponibles para descargar.")
            print("💡 Primero carga archivos que contengan datos para este instrumento.")
            return
        try:
            fecha_actual = datetime.now().strftime("%Y%m%d_%H%M%S")
            extension = 'csv' if origen == 'csv' else 'xlsx'
            nombre_archivo = f"{instrumento}_{origen}_{fecha_actual}.{extension}"
            buffer = io.BytesIO()
            if extension == 'csv':
                df.to_csv(buffer, index=False, encoding='utf-8')
                mime = "text/csv"
            else:
                df.to_excel(buffer, index=False, engine='openpyxl')
                mime = "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
            buffer.seek(0)
            b64 = base64.b64encode(buffer.read()).decode()
            html = f'''
            <div style="padding: 10px; border: 2px solid #4CAF50; border-radius: 5px; background-color: #f9f9f9;">
                <h3>📥 Descarga Lista</h3>
                <p><strong>Archivo:</strong> {nombre_archivo}</p>
                <p><strong>Filas:</strong> {len(df)} | <strong>Columnas:</strong> {len(df.columns)}</p>
                <a download="{nombre_archivo}"
                   href="data:{mime};base64,{b64}"
                   target="_blank"
                   style="display: inline-block; padding: 10px 20px; background-color: #4CAF50; color: white; text-decoration: none; border-radius: 5px; font-weight: bold;">
                   📥 Hacer clic para descargar
                </a>
            </div>
            '''
            display(HTML(html))
            print("✅ Enlace de descarga generado exitosamente.")
            print("💡 Haz clic en el enlace y elige dónde guardar el archivo.")
        except Exception as e:
            print(f"❌ Error al generar la descarga: {str(e)}")

# Conectar eventos a los botones solo una vez
boton_ver.on_click(ver_datos)
boton_descargar.on_click(descargar_datos)
upload_widget.observe(cargar_archivos, names='value')

# Mostrar la interfaz
print("🚀 Sistema de Carga de Datos de Instrumentos")
print("=" * 50)
print("📁 Formatos soportados: CSV, XLSX")
print("🔧 Instrumentos disponibles:")
for i, inst in enumerate(instrumentos, 1):
    print(f"  {i}. {inst}")
print("\n📤 Selecciona uno o más archivos para comenzar:")

display(upload_widget)
display(output)
display(widgets.HBox([instrumento_selector, origen_selector]))
display(widgets.HBox([boton_ver, boton_descargar]))
display(output_accion)

🚀 Sistema de Carga de Datos de Instrumentos
📁 Formatos soportados: CSV, XLSX
🔧 Instrumentos disponibles:
  1. puntos_fijos_mi
  2. puntos_fijos_md
  3. inclinometros
  4. asentamiento
  5. piezometros_electricos
  6. piezometros_casagrande
  7. freatimetros
  8. extensometro

📤 Selecciona uno o más archivos para comenzar:


FileUpload(value={}, accept='.csv,.xlsx', description='Subir archivos', multiple=True, style=ButtonStyle(butto…

Output()

Output()

#Version Mejorada con Selectores Botones y boton Guardar - Version 4 - Funciona Ok

In [ ]:
import pandas as pd
import io
import base64
from datetime import datetime
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets

# --- Configuración y estilos ---
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]
datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return "puntos_fijos_mi"
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# Widgets globales para mantener la UI estable
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'},
    layout=widgets.Layout(width="350px")
)
output_carga = widgets.Output()
output_tabla = widgets.Output()

instrumento_selector = widgets.Dropdown(
    options=instrumentos,
    description='Instrumento:',
    layout=widgets.Layout(width='260px')
)
origen_selector = widgets.Dropdown(
    options=['csv', 'xlsx'],
    description='Origen:',
    layout=widgets.Layout(width='160px')
)
boton_ver = widgets.Button(
    description='👁️ Ver',
    button_style='success',
    icon='eye',
    layout=widgets.Layout(width='100px')
)
boton_descargar = widgets.Button(
    description='💾 Descargar',
    button_style='info',
    icon='download',
    layout=widgets.Layout(width='130px')
)

# --- Función de carga y progreso ---
def cargar_archivos(change):
    with output_carga:
        clear_output(wait=True)
        archivos = upload_widget.value
        if not archivos:
            display(HTML("<div style='color:#b71c1c;font-weight:bold;'>⚠️ No se subió ningún archivo.</div>"))
            return

        barra_progreso = widgets.FloatProgress(
            value=0, min=0, max=100, description='Progreso:', bar_style='info',
            layout=widgets.Layout(width='80%')
        )
        etiqueta_progreso = widgets.Label(value="0% completado")
        display(barra_progreso, etiqueta_progreso)

        total = len(archivos)
        archivos_exitosos = 0

        for i, (nombre_archivo, archivo_info) in enumerate(archivos.items(), start=1):
            try:
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)
                contenido = archivo_info['content']

                if not instrumento:
                    display(HTML(f"<span style='color:#b71c1c;'>❌ Instrumento no reconocido en archivo: {nombre_archivo}</span>"))
                    continue

                if extension == 'csv':
                    try:
                        try:
                            df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                        except UnicodeDecodeError:
                            df = pd.read_csv(io.BytesIO(contenido), encoding='latin-1')
                        if not df.empty:
                            datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                            archivos_exitosos += 1
                            display(HTML(f"<span style='color:#388e3c;'>✔️ {nombre_archivo} cargado como CSV ({len(df)} filas)</span>"))
                        else:
                            display(HTML(f"<span style='color:#ffa000;'>⚠️ CSV vacío: {nombre_archivo}</span>"))
                    except Exception as e:
                        display(HTML(f"<span style='color:#b71c1c;'>❌ Error leyendo CSV {nombre_archivo}: {str(e)}</span>"))
                elif extension == 'xlsx':
                    try:
                        df = pd.read_excel(io.BytesIO(contenido))
                        if not df.empty:
                            datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                            archivos_exitosos += 1
                            display(HTML(f"<span style='color:#388e3c;'>✔️ {nombre_archivo} cargado como XLSX ({len(df)} filas)</span>"))
                        else:
                            display(HTML(f"<span style='color:#ffa000;'>⚠️ XLSX vacío: {nombre_archivo}</span>"))
                    except Exception as e:
                        display(HTML(f"<span style='color:#b71c1c;'>❌ Error leyendo XLSX {nombre_archivo}: {str(e)}</span>"))
                else:
                    display(HTML(f"<span style='color:#b71c1c;'>⚠️ Extensión no soportada: {nombre_archivo}</span>"))
            except Exception as e:
                display(HTML(f"<span style='color:#b71c1c;'>❌ Error general en {nombre_archivo}: {str(e)}</span>"))
            porcentaje = (i / total) * 100
            barra_progreso.value = porcentaje
            etiqueta_progreso.value = f"{porcentaje:.0f}% completado"

        barra_progreso.bar_style = 'success'
        etiqueta_progreso.value = f"✅ {archivos_exitosos}/{total} archivos procesados exitosamente"
        resumen = "<ul>"
        for inst in instrumentos:
            csv_count = len(datos_csv[inst])
            xlsx_count = len(datos_xlsx[inst])
            if csv_count > 0 or xlsx_count > 0:
                resumen += f"<li><b>{inst}</b>: {csv_count} filas CSV, {xlsx_count} filas XLSX</li>"
        resumen += "</ul>"
        display(HTML(f"<div style='margin-top:10px;'><b>Resumen de datos:</b>{resumen}</div>"))

# --- Función para mostrar tabla o mensaje ---
def ver_datos(b):
    with output_tabla:
        clear_output(wait=True)
        instrumento = instrumento_selector.value
        origen = origen_selector.value
        df = datos_csv[instrumento] if origen == 'csv' else datos_xlsx[instrumento]
        if df.empty:
            display(HTML("<div style='color:#b71c1c;font-weight:bold;'>⚠️ No hay datos disponibles para el instrumento y origen seleccionados.</div>"))
        else:
            display(HTML(f"<div style='margin-bottom:10px;'><b>{instrumento.replace('_',' ').title()} ({origen.upper()})</b> - <span style='color:#388e3c'>Filas: {len(df)} | Columnas: {len(df.columns)}</span></div>"))
            display(df.head(5))  # AJUSTE: solo 5 registros

# --- Función para descargar con enlace bonito ---
def descargar_datos(b):
    with output_tabla:
        clear_output(wait=True)
        instrumento = instrumento_selector.value
        origen = origen_selector.value
        df = datos_csv[instrumento] if origen == 'csv' else datos_xlsx[instrumento]
        if df.empty:
            display(HTML("<div style='color:#b71c1c;font-weight:bold;'>⚠️ No hay datos para descargar.</div>"))
            return
        fecha_actual = datetime.now().strftime("%Y%m%d_%H%M%S")
        extension = 'csv' if origen == 'csv' else 'xlsx'
        nombre_archivo = f"{instrumento}_{origen}_{fecha_actual}.{extension}"
        buffer = io.BytesIO()
        try:
            if extension == 'csv':
                df.to_csv(buffer, index=False, encoding='utf-8')
                mime = "text/csv"
            else:
                # openpyxl es necesario para xlsx, pero en Colab/Jupyter viene instalado
                df.to_excel(buffer, index=False, engine='openpyxl')
                mime = "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
            buffer.seek(0)
            b64 = base64.b64encode(buffer.read()).decode()
            # NOTA: El enlace debe estar en una sola línea sin saltos para funcionar bien
            html = f"""
            <div style='padding:12px 18px;background:#e3f2fd;border:1.5px solid #2196f3;border-radius:7px;max-width:420px;'>
                <b>Descarga lista:</b><br>
                <span style='color:#1565c0'><b>{nombre_archivo}</b></span><br>
                <a download="{nombre_archivo}" href="data:{mime};base64,{b64}" target="_blank" style="display:inline-block;margin-top:10px;padding:10px 18px;background:#2196f3;color:white;text-decoration:none;border-radius:4px;font-weight:bold;">
                   📥 Descargar archivo
                </a>
            </div>
            """
            display(HTML(html))
            display(HTML("<span style='color:#388e3c;'>✔️ Haz clic en el botón para guardar el archivo en tu PC.</span>"))
        except Exception as e:
            display(HTML(f"<span style='color:#b71c1c;'>❌ Error al preparar la descarga: {str(e)}</span>"))

# --- Conectar eventos (solo una vez) ---
upload_widget.observe(cargar_archivos, names='value')
boton_ver.on_click(ver_datos)
boton_descargar.on_click(descargar_datos)

# --- Mostrar interfaz visual limpia y separada ---
display(HTML("""
<div style='margin-bottom:15px;'>
    <h2 style='color:#1976d2;margin:0 0 4px 0;'>📈 Sistema de gestión de datos de instrumentos</h2>
    <span style='color:#555;'>Carga, visualización y descarga de archivos CSV/XLSX</span>
</div>
"""))
display(HTML("<b>1. Subí tus archivos CSV/XLSX:</b>"))
display(upload_widget)
display(output_carga)
display(HTML("<hr style='margin:20px 0 10px 0;'>"))
display(HTML("<b>2. Seleccioná instrumento y origen de datos:</b>"))
display(widgets.HBox([instrumento_selector, origen_selector, boton_ver, boton_descargar]))
display(output_tabla)

FileUpload(value={}, accept='.csv,.xlsx', description='Subir archivos', layout=Layout(width='350px'), multiple…

Output()

Output()

#Instalación de dependencias para Visualizaciones para Plotly

In [ ]:
# 💡 Esto instalará versiones compatibles
!pip install -U plotly==6.1.1 kaleido==0.2.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1


#Script Estado/Alarma Puentos Fijos

In [8]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io
from datetime import datetime

# ==== Selectores Comunes y por Instrumento ====
instrumento_dropdown = widgets.Dropdown(
    options=["Puntos Fijos", "Piezómetros Eléctricos", "Piezómetros Casagrande", "Inclinómetros",
             "Celdas de Asentamiento", "Freatímetros", "Extensómetros"],
    value="Puntos Fijos",
    description="Instrumento:",
    layout=widgets.Layout(width='300px')
)
origen_dropdown = widgets.Dropdown(
    options=["CSV", "XLSX"],
    value="CSV",
    description="Origen:",
    layout=widgets.Layout(width='200px')
)

margen_dropdown = widgets.Dropdown(description="Margen:", layout=widgets.Layout(width='320px'))
punto_dropdown = widgets.Dropdown(description="Punto Fijo:", layout=widgets.Layout(width='320px'))
anio_pf_dropdown = widgets.Dropdown(description="Año:", layout=widgets.Layout(width='180px'))

filas_dropdown = widgets.Dropdown(
    options=[10, 20, 50, 100, 500, 1000, 'Todos'],
    value=20,
    description='Filas:',
    layout=widgets.Layout(width='180px')
)

slider_widgets = {}
sliders_box = widgets.VBox()

# === Crear sliders por variable numérica ===
def crear_slider_con_botones(col, col_min, col_max, col_range):
    slider = widgets.FloatSlider(
        value=0.0,
        min=np.floor(col_min - 0.2 * col_range),
        max=np.ceil(col_max + 0.2 * col_range),
        step=0.10,
        description=f"{col}:",
        continuous_update=False,
        readout=True,
        layout=widgets.Layout(width='350px'),
        style={'description_width': '180px'}
    )
    menos_btn = widgets.Button(description='-', layout=widgets.Layout(width='32px'))
    mas_btn = widgets.Button(description='+', layout=widgets.Layout(width='32px'))

    def menos_clicked(b):
        slider.value = max(slider.value - slider.step, slider.min)
        slider.value = np.round(slider.value, 2)

    def mas_clicked(b):
        slider.value = min(slider.value + slider.step, slider.max)
        slider.value = np.round(slider.value, 2)

    menos_btn.on_click(menos_clicked)
    mas_btn.on_click(mas_clicked)

    return slider, widgets.HBox([slider, menos_btn, mas_btn])

def crear_sliders(df):
    global slider_widgets
    slider_widgets = {}
    sliders = []
    num_cols = [
        col for col in df.select_dtypes(include='number').columns
        if all(x not in col.lower() for x in ['delta', 'distancia en m', 'distancia_m', 'distancia_[m'])
        and col not in ['FECHA', 'INSTRUMENTO', 'MARGEN']
    ]
    for col in num_cols:
        col_min = df[col].min(skipna=True) if not df[col].dropna().empty else 0.0
        col_max = df[col].max(skipna=True) if not df[col].dropna().empty else col_min + 1.0
        col_range = col_max - col_min if col_max > col_min else 1.0
        slider, hbox = crear_slider_con_botones(col, col_min, col_max, col_range)
        slider_widgets[col] = slider
        sliders.append(hbox)
    sliders_box.children = sliders

def obtener_df_actual():
    origen = origen_dropdown.value
    margen = margen_dropdown.value
    if margen == "Margen Izquierda (MI)":
        return (datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]).copy()
    elif margen == "Margen Derecha (MD)":
        return (datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]).copy()
    else:
        return pd.DataFrame()

def actualizar_opciones_pf(change=None):
    origen = origen_dropdown.value
    df_mi = (datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]).copy()
    df_md = (datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]).copy()
    datasets = {"Margen Izquierda (MI)": df_mi, "Margen Derecha (MD)": df_md}
    datasets = {k: v for k, v in datasets.items() if not v.empty}
    if not datasets:
        margen_dropdown.options = []
        punto_dropdown.options = []
        anio_pf_dropdown.options = []
        sliders_box.children = []
        return
    margen_dropdown.options = list(datasets.keys())
    margen_dropdown.value = margen_dropdown.value or list(datasets.keys())[0]
    df = datasets[margen_dropdown.value].copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    punto_dropdown.options = ["Todos"] + sorted(df['INSTRUMENTO'].dropna().unique())
    punto_dropdown.value = "Todos"
    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_pf_dropdown.options = ["Todos"] + [str(a) for a in anios]
    anio_pf_dropdown.value = "Todos"
    crear_sliders(df)

origen_dropdown.observe(actualizar_opciones_pf, names='value')
margen_dropdown.observe(actualizar_opciones_pf, names='value')
actualizar_opciones_pf()

tipos_alarma = [
    ("Todos", "todos"), ("Normal 🟢", "normal"),
    ("Alerta 🟡", "alerta"), ("Peligro 🔴", "peligro"), ("Sin dato ⚪️", "sin dato")
]
alarma_selector = widgets.Dropdown(
    options=tipos_alarma, value="todos",
    description="Ver alarmas:", layout=widgets.Layout(width='180px')
)

boton_estado = widgets.Button(description="Calcular estado", button_style="warning", layout=widgets.Layout(width='160px'))
boton_descargar = widgets.Button(description="📥 Descargar XLSX", button_style="success", layout=widgets.Layout(width='180px'))

output_estado = widgets.Output()
output_tabla_alarma = widgets.Output()
output_piechart = widgets.Output()
output_descarga = widgets.Output()

def clasificar_estado_pf(row, umbrales):
    if any(pd.isna(row.get(var, np.nan)) for var in umbrales):
        return "sin dato"
    if all(umbral == 0 for umbral in umbrales.values()):
        return "normal"
    if any(row[var] > umbral for var, umbral in umbrales.items() if umbral != 0):
        return "peligro"
    if any(row[var] > 0.7 * umbral for var, umbral in umbrales.items() if umbral != 0):
        return "alerta"
    return "normal"

def emoji_alarma(estado):
    return {"normal": "🟢", "alerta": "🟡", "peligro": "🔴"}.get(estado, "⚪️")

def mostrar_tabla_alarmas(df, tipo="todos", n_filas=20):
    df = df.copy()
    df["alarma"] = df["estado"].map(emoji_alarma)
    if tipo != "todos":
        df = df[df["estado"] == tipo]
    mostrar = df if n_filas == 'Todos' else df.head(int(n_filas))
    display(HTML("<b>Tabla de alarmas:</b>"))
    columnas_mostrar = [
        "FECHA", "INSTRUMENTO", "DISTANCIA_(MM)",
        "TASA_DISTANCIA_(MM/DIA)", "AZIMUT_(GRADOS)", "estado", "alarma"
    ]
    columnas = [c for c in columnas_mostrar if c in mostrar.columns]
    display(HTML(mostrar[columnas].to_html(index=False)))

df_alarma = pd.DataFrame()

def actualizar_tabla_alarma(change=None):
    with output_tabla_alarma:
        clear_output(wait=True)
        if not df_alarma.empty:
            mostrar_tabla_alarmas(df_alarma, tipo=alarma_selector.value, n_filas=filas_dropdown.value)
        else:
            display(HTML("<b style='color:red'>No hay datos para mostrar.</b>"))

alarma_selector.observe(actualizar_tabla_alarma, names='value')
filas_dropdown.observe(actualizar_tabla_alarma, names='value')

def actualizar_piechart():
    with output_piechart:
        clear_output(wait=True)
        if not df_alarma.empty:
            estado_counts = df_alarma['estado'].value_counts()
            colores_map = {"normal": "#8BC34A", "alerta": "#FFC107", "peligro": "#F44336", "sin dato": "#BDBDBD"}
            fig = go.Figure(data=[go.Pie(
                labels=[f"{e} {emoji_alarma(e)}" for e in estado_counts.index],
                values=estado_counts.values,
                hole=.5,
                marker_colors=[colores_map.get(e, "#BDBDBD") for e in estado_counts.index]
            )])
            fig.update_layout(title_text="Distribución de estados")
            fig.show()
        else:
            display(HTML("<b style='color:red'>No hay datos para graficar.</b>"))

def calcular_estado_puntos_fijos(b=None):
    with output_estado:
        clear_output(wait=True)
        df = obtener_df_actual()
        if df.empty:
            display(HTML("<b style='color:red'>No hay datos para esta selección.</b>"))
            return
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        if anio_pf_dropdown.value != "Todos":
            df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)]
        if punto_dropdown.value != "Todos":
            df = df[df['INSTRUMENTO'] == punto_dropdown.value]
        if df.empty:
            display(HTML("<b style='color:red'>No hay datos para esta selección.</b>"))
            return
        umbrales = {var: slider.value for var, slider in slider_widgets.items()}
        df['estado'] = df.apply(lambda row: clasificar_estado_pf(row, umbrales), axis=1)
        global df_alarma
        df_alarma = df.copy()
        actualizar_tabla_alarma()
        actualizar_piechart()

def descargar_alarmas_xlsx(b=None):
    with output_descarga:
        clear_output(wait=True)
        if not df_alarma.empty:
            fecha_str = datetime.now().strftime("%d-%m-%y")
            nombre_archivo = f"alarmas_{fecha_str}.xlsx"
            buffer = io.BytesIO()
            columnas_exportar = [
                "FECHA", "INSTRUMENTO", "DISTANCIA_(MM)",
                "TASA_DISTANCIA_(MM/DIA)", "AZIMUT_(GRADOS)", "estado"
            ]
            columnas = [c for c in columnas_exportar if c in df_alarma.columns]
            df_alarma[columnas].to_excel(buffer, index=False)
            buffer.seek(0)
            import base64
            b64 = base64.b64encode(buffer.read()).decode()
            href = f'<a download="{nombre_archivo}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}">📥 Descargar {nombre_archivo}</a>'
            display(HTML(href))
        else:
            display(HTML("<b style='color:red'>No hay datos para descargar.</b>"))

boton_estado.on_click(calcular_estado_puntos_fijos)
boton_descargar.on_click(descargar_alarmas_xlsx)

pf_controles = widgets.VBox([
    widgets.HBox([origen_dropdown, margen_dropdown, punto_dropdown, anio_pf_dropdown]),
    widgets.HTML("<b>Ajustá los umbrales para clasificar alarmas:</b>"),
    sliders_box,
    widgets.HBox([filas_dropdown, alarma_selector, boton_estado, boton_descargar]),
    output_descarga
])

display(widgets.HTML("<h3 style='color:#1866a3'>Estado/alarma automático por instrumento</h3>"))
display(instrumento_dropdown)
display_area = widgets.VBox([pf_controles, output_estado, output_tabla_alarma, output_piechart])
display(display_area)


HTML(value="<h3 style='color:#1866a3'>Estado/alarma automático por instrumento</h3>")

Dropdown(description='Instrumento:', layout=Layout(width='300px'), options=('Puntos Fijos', 'Piezómetros Eléct…

#Script Estado/Alarma Piezómetros Eléctricos

In [9]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io
from datetime import datetime
import base64

# === Selectores ===
origen_dropdown_pe = widgets.Dropdown(
    options=["CSV", "XLSX"],
    value="CSV",
    description="Origen:",
    layout=widgets.Layout(width='180px')
)
progresiva_dropdown_pe = widgets.Dropdown(description="Progresiva:", layout=widgets.Layout(width='220px'))
piezometro_dropdown_pe = widgets.Dropdown(description="Piezómetro:", layout=widgets.Layout(width='220px'))
anio_pe_dropdown = widgets.Dropdown(description="Año:", layout=widgets.Layout(width='160px'))
filas_dropdown_pe = widgets.Dropdown(
    options=[10, 20, 50, 100, 500, 1000, 'Todos'],
    value=20,
    description='Filas:',
    layout=widgets.Layout(width='160px')
)

slider_widgets_pe = {}
sliders_box_pe = widgets.VBox()

# === Crear sliders ===
def crear_slider_con_botones_pe(col, col_min, col_max, col_range):
    slider = widgets.FloatSlider(
        value=0.0,
        min=np.floor(col_min - 0.2 * col_range),
        max=np.ceil(col_max + 0.2 * col_range),
        step=0.1,
        description=f"{col}:",
        continuous_update=False,
        readout=True,
        readout_format='.2f',
        layout=widgets.Layout(width='350px'),
        style={'description_width': '180px'}
    )
    menos_btn = widgets.Button(description='-', layout=widgets.Layout(width='32px'))
    mas_btn = widgets.Button(description='+', layout=widgets.Layout(width='32px'))

    def menos_clicked(b):
        slider.value = max(slider.value - slider.step, slider.min)
        slider.value = np.round(slider.value, 2)

    def mas_clicked(b):
        slider.value = min(slider.value + slider.step, slider.max)
        slider.value = np.round(slider.value, 2)

    menos_btn.on_click(menos_clicked)
    mas_btn.on_click(mas_clicked)

    return slider, widgets.HBox([slider, menos_btn, mas_btn])


def crear_sliders_pe(df):
    global slider_widgets_pe
    slider_widgets_pe = {}
    sliders = []
    incluir = ['COTA_RIO_(M.S.N.M)', 'MCA_2_(FACTOR_G_Y_K)', 'COTA_NF']
    num_cols = [col for col in df.columns if col in incluir]

    for col in num_cols:
        col_numeric = pd.to_numeric(df[col], errors='coerce')
        if col == 'COTA_RIO_(M.S.N.M)':
            col_min, col_max = 0, 130
        else:
            col_min = col_numeric.min(skipna=True) if not col_numeric.dropna().empty else 0.0
            col_max = col_numeric.max(skipna=True) if not col_numeric.dropna().empty else col_min + 1.0
        col_range = col_max - col_min if col_max > col_min else 1.0
        slider, hbox = crear_slider_con_botones_pe(col, col_min, col_max, col_range)
        slider_widgets_pe[col] = slider
        sliders.append(hbox)

    sliders_box_pe.children = sliders

# === Cargar DataFrame ===
def obtener_df_actual_pe():
    origen = origen_dropdown_pe.value
    return (datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]).copy()

# === Actualizar opciones de selectores ===
def actualizar_opciones_pe(change=None):
    df = obtener_df_actual_pe()
    if df.empty:
        progresiva_dropdown_pe.options = []
        piezometro_dropdown_pe.options = []
        anio_pe_dropdown.options = []
        sliders_box_pe.children = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresivas = sorted(df['PROGRESIVA'].dropna().unique())
    progresiva_dropdown_pe.options = ["Todas"] + progresivas
    progresiva_dropdown_pe.value = "Todas"
    actualizar_piezometros_pe()
    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_pe_dropdown.options = ["Todos"] + [str(a) for a in anios]
    anio_pe_dropdown.value = "Todos"
    crear_sliders_pe(df)

def actualizar_piezometros_pe(change=None):
    df = obtener_df_actual_pe()
    if df.empty:
        piezometro_dropdown_pe.options = []
        return
    prog = progresiva_dropdown_pe.value
    if prog == "Todas":
        piezos = sorted(df['PIEZOMETRO'].dropna().unique())
    else:
        piezos = sorted(df[df['PROGRESIVA'] == prog]['PIEZOMETRO'].dropna().unique())
    piezometro_dropdown_pe.options = ["Todos"] + list(piezos)
    piezometro_dropdown_pe.value = "Todos"

progresiva_dropdown_pe.observe(actualizar_piezometros_pe, names='value')
origen_dropdown_pe.observe(actualizar_opciones_pe, names='value')
actualizar_opciones_pe()

# === Alarmas ===
tipos_alarma_pe = [("Todos", "todos"), ("Normal 🟢", "normal"), ("Alerta 🟡", "alerta"), ("Peligro 🔴", "peligro"), ("Sin dato ⚪️", "sin dato")]
alarma_selector_pe = widgets.Dropdown(
    options=tipos_alarma_pe, value="todos",
    description="Ver alarmas:", layout=widgets.Layout(width='160px')
)

boton_estado_pe = widgets.Button(description="Calcular estado", button_style="warning", layout=widgets.Layout(width='150px'))
boton_descargar_pe = widgets.Button(description="📥 Descargar XLSX", button_style="success", layout=widgets.Layout(width='180px'))

output_estado_pe = widgets.Output()
output_tabla_pe = widgets.Output()
output_piechart_pe = widgets.Output()
output_descarga_pe = widgets.Output()

def clasificar_estado_pe(row, umbrales):
    for var in umbrales:
        valor = pd.to_numeric(row.get(var, np.nan), errors='coerce')
        if pd.isna(valor):
            return "sin dato"
    if all(umbral == 0 for umbral in umbrales.values()):
        return "normal"
    if any(pd.to_numeric(row[var], errors='coerce') > umbral for var, umbral in umbrales.items() if umbral != 0):
        return "peligro"
    if any(pd.to_numeric(row[var], errors='coerce') > 0.7 * umbral for var, umbral in umbrales.items() if umbral != 0):
        return "alerta"
    return "normal"

def emoji_alarma_pe(estado):
    return {"normal": "🟢", "alerta": "🟡", "peligro": "🔴"}.get(estado, "⚪️")

def mostrar_tabla_pe(df, tipo="todos", n_filas=20):
    df = df.copy()
    df["alarma"] = df["estado"].map(emoji_alarma_pe)
    if tipo != "todos":
        df = df[df["estado"] == tipo]
    mostrar = df if n_filas == 'Todos' else df.head(int(n_filas))
    display(HTML("<b>Tabla de alarmas - Piezómetros Eléctricos:</b>"))
    columnas = [
        "FECHA", "PROGRESIVA", "PIEZOMETRO", "COTA_RIO_(M.S.N.M)",
        "MCA_2_(FACTOR_G_Y_K)", "COTA_NF", "PRECIPITACIONES_(MM)", "estado", "alarma"
    ]
    columnas = [col for col in columnas if col in mostrar.columns]
    display(HTML(mostrar[columnas].to_html(index=False)))

df_pe_alarma = pd.DataFrame()

def actualizar_tabla_pe(change=None):
    with output_tabla_pe:
        clear_output(wait=True)
        if not df_pe_alarma.empty:
            mostrar_tabla_pe(df_pe_alarma, tipo=alarma_selector_pe.value, n_filas=filas_dropdown_pe.value)
        else:
            display(HTML("<b style='color:red'>No hay datos para mostrar.</b>"))

alarma_selector_pe.observe(actualizar_tabla_pe, names='value')
filas_dropdown_pe.observe(actualizar_tabla_pe, names='value')

def actualizar_piechart_pe():
    with output_piechart_pe:
        clear_output(wait=True)
        if not df_pe_alarma.empty:
            estado_counts = df_pe_alarma['estado'].value_counts()
            colores_map = {"normal": "#8BC34A", "alerta": "#FFC107", "peligro": "#F44336", "sin dato": "#BDBDBD"}
            fig = go.Figure(data=[go.Pie(
                labels=[f"{e} {emoji_alarma_pe(e)}" for e in estado_counts.index],
                values=estado_counts.values,
                hole=.5,
                marker_colors=[colores_map.get(e, "#BDBDBD") for e in estado_counts.index]
            )])
            fig.update_layout(title_text="Distribución de estados - Piezómetros Eléctricos")
            fig.show()
        else:
            display(HTML("<b style='color:red'>No hay datos para graficar.</b>"))

def calcular_estado_pe(b=None):
    with output_estado_pe:
        clear_output(wait=True)
        df = obtener_df_actual_pe()
        if df.empty:
            display(HTML("<b style='color:red'>No hay datos para esta selección.</b>"))
            return
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        if anio_pe_dropdown.value != "Todos":
            try:
              anio_seleccionado = int(anio_pe_dropdown.value)
              df = df[df['FECHA'].dt.year == anio_seleccionado]
            except Exception as e:
              display(HTML(f"<b style='color:red'>Error al filtrar por año: {e}</b>"))

        if progresiva_dropdown_pe.value != "Todas":
            df = df[df['PROGRESIVA'] == progresiva_dropdown_pe.value]
        if piezometro_dropdown_pe.value != "Todos":
            df = df[df['PIEZOMETRO'] == piezometro_dropdown_pe.value]
        if df.empty:
            display(HTML("<b style='color:red'>No hay datos para esta selección.</b>"))
            return
        umbrales = {var: slider.value for var, slider in slider_widgets_pe.items()}
        df['estado'] = df.apply(lambda row: clasificar_estado_pe(row, umbrales), axis=1)
        global df_pe_alarma
        df_pe_alarma = df.copy()
        actualizar_tabla_pe()
        actualizar_piechart_pe()

def descargar_pe_xlsx(b=None):
    with output_descarga_pe:
        clear_output(wait=True)
        if not df_pe_alarma.empty:
            fecha_str = datetime.now().strftime("%d-%m-%y")
            nombre_archivo = f"alarmas_piezometros_{fecha_str}.xlsx"
            buffer = io.BytesIO()
            columnas_exportar = [
                "FECHA", "PROGRESIVA", "PIEZOMETRO", "COTA_RIO_(M.S.N.M)",
                "MCA_2_(FACTOR_G_Y_K)", "COTA_NF", "PRECIPITACIONES_(MM)", "estado"
            ]
            columnas = [col for col in columnas_exportar if col in df_pe_alarma.columns]
            df_pe_alarma[columnas].to_excel(buffer, index=False)
            buffer.seek(0)
            b64 = base64.b64encode(buffer.read()).decode()
            href = f'<a download="{nombre_archivo}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}">📥 Descargar {nombre_archivo}</a>'
            display(HTML(href))
        else:
            display(HTML("<b style='color:red'>No hay datos para descargar.</b>"))

boton_estado_pe.on_click(calcular_estado_pe)
boton_descargar_pe.on_click(descargar_pe_xlsx)

# === Display final ===
pe_controles = widgets.VBox([
    widgets.HBox([origen_dropdown_pe, progresiva_dropdown_pe, piezometro_dropdown_pe, anio_pe_dropdown]),
    widgets.HTML("<b>Ajustá los umbrales para clasificar alarmas:</b>"),
    sliders_box_pe,
    widgets.HBox([filas_dropdown_pe, alarma_selector_pe, boton_estado_pe, boton_descargar_pe]),
    output_descarga_pe
])

display(widgets.HTML("<h3 style='color:#1866a3'>Estado/alarma automático - Piezómetros Eléctricos</h3>"))
display(pe_controles, output_estado_pe, output_tabla_pe, output_piechart_pe)


HTML(value="<h3 style='color:#1866a3'>Estado/alarma automático - Piezómetros Eléctricos</h3>")

Output()

Output()

Output()

#Script Estado/Alarma Piezómetros Casagrande

In [10]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io
from datetime import datetime
import base64

# === Selectores ===
origen_dropdown_pcg = widgets.Dropdown(
    options=["CSV", "XLSX"],
    value="CSV",
    description="Origen:",
    layout=widgets.Layout(width='180px')
)
margen_dropdown_pcg = widgets.Dropdown(description="Margen:", layout=widgets.Layout(width='180px'))
piezometro_dropdown_pcg = widgets.Dropdown(description="Piezómetro:", layout=widgets.Layout(width='220px'))
anio_pcg_dropdown = widgets.Dropdown(description="Año:", layout=widgets.Layout(width='160px'))
filas_dropdown_pcg = widgets.Dropdown(
    options=[10, 20, 50, 100, 500, 1000, 'Todos'],
    value=20,
    description='Filas:',
    layout=widgets.Layout(width='160px')
)

slider_widgets_pcg = {}
sliders_box_pcg = widgets.VBox()

# === Crear sliders ===
def crear_slider_con_botones_pcg(col, col_min, col_max, col_range):
    slider = widgets.FloatSlider(
        value=0.0,
        min=np.floor(col_min - 0.2 * col_range),
        max=np.ceil(col_max + 0.2 * col_range),
        step=0.1,
        description=f"{col}:",
        continuous_update=False,
        readout=True,
        readout_format='.2f',
        layout=widgets.Layout(width='350px'),
        style={'description_width': '180px'}
    )
    menos_btn = widgets.Button(description='-', layout=widgets.Layout(width='32px'))
    mas_btn = widgets.Button(description='+', layout=widgets.Layout(width='32px'))

    def menos_clicked(b):
        slider.value = max(slider.value - slider.step, slider.min)
        slider.value = np.round(slider.value, 2)

    def mas_clicked(b):
        slider.value = min(slider.value + slider.step, slider.max)
        slider.value = np.round(slider.value, 2)

    menos_btn.on_click(menos_clicked)
    mas_btn.on_click(mas_clicked)

    return slider, widgets.HBox([slider, menos_btn, mas_btn])

def crear_sliders_pcg(df):
    global slider_widgets_pcg
    slider_widgets_pcg = {}
    sliders = []
    incluir = ['MCA_2_(FACTOR_G_Y_K)', 'COTA_NF']
    num_cols = [col for col in df.columns if col in incluir]

    for col in num_cols:
        col_numeric = pd.to_numeric(df[col], errors='coerce')
        col_min = col_numeric.min(skipna=True) if not col_numeric.dropna().empty else 0.0
        col_max = col_numeric.max(skipna=True) if not col_numeric.dropna().empty else col_min + 1.0
        col_range = col_max - col_min if col_max > col_min else 1.0
        slider, hbox = crear_slider_con_botones_pcg(col, col_min, col_max, col_range)
        slider_widgets_pcg[col] = slider
        sliders.append(hbox)

    sliders_box_pcg.children = sliders

# === Cargar DataFrame ===
def obtener_df_actual_pcg():
    origen = origen_dropdown_pcg.value
    return (datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]).copy()

# === Actualizar opciones de selectores ===
def actualizar_opciones_pcg(change=None):
    df = obtener_df_actual_pcg()
    if df.empty:
        margen_dropdown_pcg.options = []
        piezometro_dropdown_pcg.options = []
        anio_pcg_dropdown.options = []
        sliders_box_pcg.children = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    margenes = sorted(df['MARGEN'].dropna().unique())
    margen_dropdown_pcg.options = ["Todos"] + margenes
    margen_dropdown_pcg.value = "Todos"
    actualizar_piezometros_pcg()
    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_pcg_dropdown.options = ["Todos"] + [str(a) for a in anios]
    anio_pcg_dropdown.value = "Todos"
    crear_sliders_pcg(df)

def actualizar_piezometros_pcg(change=None):
    df = obtener_df_actual_pcg()
    if df.empty:
        piezometro_dropdown_pcg.options = []
        return
    margen = margen_dropdown_pcg.value
    if margen == "Todos":
        piezos = sorted(df['PIEZOMETRO'].dropna().unique())
    else:
        piezos = sorted(df[df['MARGEN'] == margen]['PIEZOMETRO'].dropna().unique())
    piezometro_dropdown_pcg.options = ["Todos"] + list(piezos)
    piezometro_dropdown_pcg.value = "Todos"

margen_dropdown_pcg.observe(actualizar_piezometros_pcg, names='value')
origen_dropdown_pcg.observe(actualizar_opciones_pcg, names='value')
actualizar_opciones_pcg()

# === Alarmas ===
tipos_alarma_pcg = [("Todos", "todos"), ("Normal 🟢", "normal"), ("Alerta 🟡", "alerta"), ("Peligro 🔴", "peligro"), ("Sin dato ⚪️", "sin dato")]
alarma_selector_pcg = widgets.Dropdown(
    options=tipos_alarma_pcg, value="todos",
    description="Ver alarmas:", layout=widgets.Layout(width='160px')
)

boton_estado_pcg = widgets.Button(description="Calcular estado", button_style="warning", layout=widgets.Layout(width='150px'))
boton_descargar_pcg = widgets.Button(description="📥 Descargar XLSX", button_style="success", layout=widgets.Layout(width='180px'))

output_estado_pcg = widgets.Output()
output_tabla_pcg = widgets.Output()
output_piechart_pcg = widgets.Output()
output_descarga_pcg = widgets.Output()

def clasificar_estado_pcg(row, umbrales):
    for var in umbrales:
        valor = pd.to_numeric(row.get(var, np.nan), errors='coerce')
        if pd.isna(valor):
            return "sin dato"
    if all(umbral == 0 for umbral in umbrales.values()):
        return "normal"
    if any(pd.to_numeric(row[var], errors='coerce') > umbral for var, umbral in umbrales.items() if umbral != 0):
        return "peligro"
    if any(pd.to_numeric(row[var], errors='coerce') > 0.7 * umbral for var, umbral in umbrales.items() if umbral != 0):
        return "alerta"
    return "normal"

def emoji_alarma_pcg(estado):
    return {"normal": "🟢", "alerta": "🟡", "peligro": "🔴"}.get(estado, "⚪️")

def mostrar_tabla_pcg(df, tipo="todos", n_filas=20):
    df = df.copy()
    df["alarma"] = df["estado"].map(emoji_alarma_pcg)
    if tipo != "todos":
        df = df[df["estado"] == tipo]
    mostrar = df if n_filas == 'Todos' else df.head(int(n_filas))
    display(HTML("<b>Tabla de alarmas - Piezómetros Casagrande:</b>"))
    columnas = [
        "FECHA", "MARGEN", "PIEZOMETRO",
        "MCA_2_(FACTOR_G_Y_K)", "COTA_NF", "estado", "alarma"
    ]
    columnas = [col for col in columnas if col in mostrar.columns]
    display(HTML(mostrar[columnas].to_html(index=False)))

df_pcg_alarma = pd.DataFrame()

def actualizar_tabla_pcg(change=None):
    with output_tabla_pcg:
        clear_output(wait=True)
        if not df_pcg_alarma.empty:
            mostrar_tabla_pcg(df_pcg_alarma, tipo=alarma_selector_pcg.value, n_filas=filas_dropdown_pcg.value)
        else:
            display(HTML("<b style='color:red'>No hay datos para mostrar.</b>"))

alarma_selector_pcg.observe(actualizar_tabla_pcg, names='value')
filas_dropdown_pcg.observe(actualizar_tabla_pcg, names='value')

def actualizar_piechart_pcg():
    with output_piechart_pcg:
        clear_output(wait=True)
        if not df_pcg_alarma.empty:
            estado_counts = df_pcg_alarma['estado'].value_counts()
            colores_map = {"normal": "#8BC34A", "alerta": "#FFC107", "peligro": "#F44336", "sin dato": "#BDBDBD"}
            fig = go.Figure(data=[go.Pie(
                labels=[f"{e} {emoji_alarma_pcg(e)}" for e in estado_counts.index],
                values=estado_counts.values,
                hole=.5,
                marker_colors=[colores_map.get(e, "#BDBDBD") for e in estado_counts.index]
            )])
            fig.update_layout(title_text="Distribución de estados - Piezómetros Casagrande")
            fig.show()
        else:
            display(HTML("<b style='color:red'>No hay datos para graficar.</b>"))

def calcular_estado_pcg(b=None):
    with output_estado_pcg:
        clear_output(wait=True)
        df = obtener_df_actual_pcg()
        if df.empty:
            display(HTML("<b style='color:red'>No hay datos para esta selección.</b>"))
            return
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        if anio_pcg_dropdown.value != "Todos":
            try:
                anio_sel = int(anio_pcg_dropdown.value)
                df = df[df['FECHA'].dt.year == anio_sel]
            except:
                pass
        if margen_dropdown_pcg.value != "Todos":
            df = df[df['MARGEN'] == margen_dropdown_pcg.value]
        if piezometro_dropdown_pcg.value != "Todos":
            df = df[df['PIEZOMETRO'] == piezometro_dropdown_pcg.value]
        if df.empty:
            display(HTML("<b style='color:red'>No hay datos para esta selección.</b>"))
            return
        umbrales = {var: slider.value for var, slider in slider_widgets_pcg.items()}
        df['estado'] = df.apply(lambda row: clasificar_estado_pcg(row, umbrales), axis=1)
        global df_pcg_alarma
        df_pcg_alarma = df.copy()
        actualizar_tabla_pcg()
        actualizar_piechart_pcg()

def descargar_pcg_xlsx(b=None):
    with output_descarga_pcg:
        clear_output(wait=True)
        if not df_pcg_alarma.empty:
            fecha_str = datetime.now().strftime("%d-%m-%y")
            nombre_archivo = f"alarmas_piezometros_casagrande_{fecha_str}.xlsx"
            buffer = io.BytesIO()
            columnas_exportar = [
                "FECHA", "MARGEN", "PIEZOMETRO",
                "MCA_2_(FACTOR_G_Y_K)", "COTA_NF", "estado"
            ]
            columnas = [col for col in columnas_exportar if col in df_pcg_alarma.columns]
            df_pcg_alarma[columnas].to_excel(buffer, index=False)
            buffer.seek(0)
            b64 = base64.b64encode(buffer.read()).decode()
            href = f'<a download="{nombre_archivo}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}">📥 Descargar {nombre_archivo}</a>'
            display(HTML(href))
        else:
            display(HTML("<b style='color:red'>No hay datos para descargar.</b>"))

boton_estado_pcg.on_click(calcular_estado_pcg)
boton_descargar_pcg.on_click(descargar_pcg_xlsx)

# === Display final ===
pcg_controles = widgets.VBox([
    widgets.HBox([origen_dropdown_pcg, margen_dropdown_pcg, piezometro_dropdown_pcg, anio_pcg_dropdown]),
    widgets.HTML("<b>Ajustá los umbrales para clasificar alarmas:</b>"),
    sliders_box_pcg,
    widgets.HBox([filas_dropdown_pcg, alarma_selector_pcg, boton_estado_pcg, boton_descargar_pcg]),
    output_descarga_pcg
])

display(widgets.HTML("<h3 style='color:#1866a3'>Estado/alarma automático - Piezómetros Casagrande</h3>"))
display(pcg_controles, output_estado_pcg, output_tabla_pcg, output_piechart_pcg)


HTML(value="<h3 style='color:#1866a3'>Estado/alarma automático - Piezómetros Casagrande</h3>")

Output()

Output()

Output()

In [11]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io
from datetime import datetime
import base64

# === Selectores ===
origen_dropdown_in = widgets.Dropdown(
    options=["CSV", "XLSX"],
    value="CSV",
    description="Origen:",
    layout=widgets.Layout(width='180px')
)
inclinometro_dropdown_in = widgets.Dropdown(description="Inclinómetro:", layout=widgets.Layout(width='220px'))
sonda_dropdown_in = widgets.Dropdown(description="Nro Sonda:", layout=widgets.Layout(width='180px'))
anio_in_dropdown = widgets.Dropdown(description="Año:", layout=widgets.Layout(width='140px'))
filas_dropdown_in = widgets.Dropdown(
    options=[10, 20, 50, 100, 500, 1000, 'Todos'],
    value=20,
    description='Filas:',
    layout=widgets.Layout(width='140px')
)

slider_widgets_in = {}
sliders_box_in = widgets.VBox()

# === Crear sliders ===
def crear_slider_con_botones_in(col, col_min, col_max, col_range):
    slider = widgets.FloatSlider(
        value=0.0,
        min=np.floor(col_min - 0.2 * col_range),
        max=np.ceil(col_max + 0.2 * col_range),
        step=0.1,
        description=f"{col}:",
        continuous_update=False,
        readout=True,
        readout_format='.2f',
        layout=widgets.Layout(width='350px'),
        style={'description_width': '180px'}
    )
    menos_btn = widgets.Button(description='-', layout=widgets.Layout(width='32px'))
    mas_btn = widgets.Button(description='+', layout=widgets.Layout(width='32px'))

    def menos_clicked(b):
        slider.value = max(slider.value - slider.step, slider.min)
        slider.value = np.round(slider.value, 2)

    def mas_clicked(b):
        slider.value = min(slider.value + slider.step, slider.max)
        slider.value = np.round(slider.value, 2)

    menos_btn.on_click(menos_clicked)
    mas_btn.on_click(mas_clicked)

    return slider, widgets.HBox([slider, menos_btn, mas_btn])

def crear_sliders_in(df):
    global slider_widgets_in
    slider_widgets_in = {}
    sliders = []
    incluir = ['A+', 'A-', 'B+', 'B-']
    num_cols = [col for col in df.columns if col in incluir]

    for col in num_cols:
        col_numeric = pd.to_numeric(df[col], errors='coerce')
        col_min = col_numeric.min(skipna=True) if not col_numeric.dropna().empty else 0.0
        col_max = col_numeric.max(skipna=True) if not col_numeric.dropna().empty else col_min + 1.0
        col_range = col_max - col_min if col_max > col_min else 1.0
        slider, hbox = crear_slider_con_botones_in(col, col_min, col_max, col_range)
        slider_widgets_in[col] = slider
        sliders.append(hbox)

    sliders_box_in.children = sliders

# === Cargar DataFrame ===
def obtener_df_actual_in():
    origen = origen_dropdown_in.value
    return (datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]).copy()

# === Actualizar opciones de selectores ===
def actualizar_opciones_in(change=None):
    df = obtener_df_actual_in()
    if df.empty:
        inclinometro_dropdown_in.options = []
        sonda_dropdown_in.options = []
        anio_in_dropdown.options = []
        sliders_box_in.children = []
        return
    df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
    inclinos = sorted(df['Inclinometro'].dropna().unique())
    inclinometro_dropdown_in.options = ["Todos"] + inclinos
    inclinometro_dropdown_in.value = "Todos"
    actualizar_sondas_in()
    anios = sorted(df['Fecha'].dt.year.dropna().unique())
    anio_in_dropdown.options = ["Todos"] + [str(a) for a in anios]
    anio_in_dropdown.value = "Todos"
    crear_sliders_in(df)

def actualizar_sondas_in(change=None):
    df = obtener_df_actual_in()
    if df.empty:
        sonda_dropdown_in.options = []
        return
    inclino = inclinometro_dropdown_in.value
    if inclino == "Todos":
        sondas = sorted(df['Nro_Sonda'].dropna().unique())
    else:
        sondas = sorted(df[df['Inclinometro'] == inclino]['Nro_Sonda'].dropna().unique())
    sonda_dropdown_in.options = ["Todos"] + list(sondas)
    sonda_dropdown_in.value = "Todos"

inclinometro_dropdown_in.observe(actualizar_sondas_in, names='value')
origen_dropdown_in.observe(actualizar_opciones_in, names='value')
actualizar_opciones_in()

# === Alarmas ===
tipos_alarma_in = [("Todos", "todos"), ("Normal 🟢", "normal"), ("Alerta 🟡", "alerta"), ("Peligro 🔴", "peligro"), ("Sin dato ⚪️", "sin dato")]
alarma_selector_in = widgets.Dropdown(
    options=tipos_alarma_in, value="todos",
    description="Ver alarmas:", layout=widgets.Layout(width='160px')
)

boton_estado_in = widgets.Button(description="Calcular estado", button_style="warning", layout=widgets.Layout(width='150px'))
boton_descargar_in = widgets.Button(description="📥 Descargar XLSX", button_style="success", layout=widgets.Layout(width='180px'))

output_estado_in = widgets.Output()
output_tabla_in = widgets.Output()
output_piechart_in = widgets.Output()
output_descarga_in = widgets.Output()

def clasificar_estado_in(row, umbrales):
    for var in umbrales:
        valor = pd.to_numeric(row.get(var, np.nan), errors='coerce')
        if pd.isna(valor):
            return "sin dato"
    if all(umbral == 0 for umbral in umbrales.values()):
        return "normal"
    if any(pd.to_numeric(row[var], errors='coerce') > umbral for var, umbral in umbrales.items() if umbral != 0):
        return "peligro"
    if any(pd.to_numeric(row[var], errors='coerce') > 0.7 * umbral for var, umbral in umbrales.items() if umbral != 0):
        return "alerta"
    return "normal"

def emoji_alarma_in(estado):
    return {"normal": "🟢", "alerta": "🟡", "peligro": "🔴"}.get(estado, "⚪️")

def mostrar_tabla_in(df, tipo="todos", n_filas=20):
    df = df.copy()
    df["alarma"] = df["estado"].map(emoji_alarma_in)
    if tipo != "todos":
        df = df[df["estado"] == tipo]
    mostrar = df if n_filas == 'Todos' else df.head(int(n_filas))
    display(HTML("<b>Tabla de alarmas - Inclinómetros:</b>"))
    columnas = ["Fecha", "Inclinometro", "Profundidad", "A+", "A-", "B+", "B-", "Nro_Sonda", "estado", "alarma"]
    columnas = [col for col in columnas if col in mostrar.columns]
    display(HTML(mostrar[columnas].to_html(index=False)))

df_in_alarma = pd.DataFrame()

def actualizar_tabla_in(change=None):
    with output_tabla_in:
        clear_output(wait=True)
        if not df_in_alarma.empty:
            mostrar_tabla_in(df_in_alarma, tipo=alarma_selector_in.value, n_filas=filas_dropdown_in.value)
        else:
            display(HTML("<b style='color:red'>No hay datos para mostrar.</b>"))

alarma_selector_in.observe(actualizar_tabla_in, names='value')
filas_dropdown_in.observe(actualizar_tabla_in, names='value')

def actualizar_piechart_in():
    with output_piechart_in:
        clear_output(wait=True)
        if not df_in_alarma.empty:
            estado_counts = df_in_alarma['estado'].value_counts()
            colores_map = {"normal": "#8BC34A", "alerta": "#FFC107", "peligro": "#F44336", "sin dato": "#BDBDBD"}
            fig = go.Figure(data=[go.Pie(
                labels=[f"{e} {emoji_alarma_in(e)}" for e in estado_counts.index],
                values=estado_counts.values,
                hole=.5,
                marker_colors=[colores_map.get(e, "#BDBDBD") for e in estado_counts.index]
            )])
            fig.update_layout(title_text="Distribución de estados - Inclinómetros")
            fig.show()
        else:
            display(HTML("<b style='color:red'>No hay datos para graficar.</b>"))

def calcular_estado_in(b=None):
    with output_estado_in:
        clear_output(wait=True)
        df = obtener_df_actual_in()
        if df.empty:
            display(HTML("<b style='color:red'>No hay datos para esta selección.</b>"))
            return
        df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
        if anio_in_dropdown.value != "Todos":
            try:
                anio_sel = int(anio_in_dropdown.value)
                df = df[df['Fecha'].dt.year == anio_sel]
            except:
                pass
        if inclinometro_dropdown_in.value != "Todos":
            df = df[df['Inclinometro'] == inclinometro_dropdown_in.value]
        if sonda_dropdown_in.value != "Todos":
            df = df[df['Nro_Sonda'] == sonda_dropdown_in.value]
        if df.empty:
            display(HTML("<b style='color:red'>No hay datos para esta selección.</b>"))
            return
        umbrales = {var: slider.value for var, slider in slider_widgets_in.items()}
        df['estado'] = df.apply(lambda row: clasificar_estado_in(row, umbrales), axis=1)
        global df_in_alarma
        df_in_alarma = df.copy()
        actualizar_tabla_in()
        actualizar_piechart_in()

def descargar_in_xlsx(b=None):
    with output_descarga_in:
        clear_output(wait=True)
        if not df_in_alarma.empty:
            fecha_str = datetime.now().strftime("%d-%m-%y")
            nombre_archivo = f"alarmas_inclinometros_{fecha_str}.xlsx"
            buffer = io.BytesIO()
            columnas_exportar = ["Fecha", "Inclinometro", "Profundidad", "A+", "A-", "B+", "B-", "Nro_Sonda", "estado"]
            columnas = [col for col in columnas_exportar if col in df_in_alarma.columns]
            df_in_alarma[columnas].to_excel(buffer, index=False)
            buffer.seek(0)
            b64 = base64.b64encode(buffer.read()).decode()
            href = f'<a download="{nombre_archivo}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}">📥 Descargar {nombre_archivo}</a>'
            display(HTML(href))
        else:
            display(HTML("<b style='color:red'>No hay datos para descargar.</b>"))

boton_estado_in.on_click(calcular_estado_in)
boton_descargar_in.on_click(descargar_in_xlsx)

# === Display final ===
in_controles = widgets.VBox([
    widgets.HBox([origen_dropdown_in, inclinometro_dropdown_in, sonda_dropdown_in, anio_in_dropdown]),
    widgets.HTML("<b>Ajustá los umbrales para clasificar alarmas:</b>"),
    sliders_box_in,
    widgets.HBox([filas_dropdown_in, alarma_selector_in, boton_estado_in, boton_descargar_in]),
    output_descarga_in
])

display(widgets.HTML("<h3 style='color:#1866a3'>Estado/alarma automático - Inclinómetros</h3>"))
display(in_controles, output_estado_in, output_tabla_in, output_piechart_in)


HTML(value="<h3 style='color:#1866a3'>Estado/alarma automático - Inclinómetros</h3>")

Output()

Output()

Output()

#Script Estado/Alerta Celdas de Asentamiento

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io
from datetime import datetime
import base64

# === Selectores ===
origen_dropdown_as = widgets.Dropdown(
    options=["CSV", "XLSX"],
    value="CSV",
    description="Origen:",
    layout=widgets.Layout(width='180px')
)
progresiva_dropdown_as = widgets.Dropdown(description="Progresiva:", layout=widgets.Layout(width='200px'))
celda_dropdown_as = widgets.Dropdown(description="Celda:", layout=widgets.Layout(width='200px'))
anio_as_dropdown = widgets.Dropdown(description="Año:", layout=widgets.Layout(width='120px'))
filas_dropdown_as = widgets.Dropdown(
    options=[10, 20, 50, 100, 500, 1000, 'Todos'],
    value=20,
    description='Filas:',
    layout=widgets.Layout(width='120px')
)

slider_widgets_as = {}
sliders_box_as = widgets.VBox()

# === Crear sliders ===
def crear_slider_con_botones_as(col, col_min, col_max, col_range):
    slider = widgets.FloatSlider(
        value=0.0,
        min=np.floor(col_min - 0.2 * col_range),
        max=np.ceil(col_max + 0.2 * col_range),
        step=0.1,
        description=f"{col}:",
        continuous_update=False,
        readout=True,
        readout_format='.2f',
        layout=widgets.Layout(width='350px'),
        style={'description_width': '180px'}
    )
    menos_btn = widgets.Button(description='-', layout=widgets.Layout(width='32px'))
    mas_btn = widgets.Button(description='+', layout=widgets.Layout(width='32px'))

    def menos_clicked(b):
        slider.value = max(slider.value - slider.step, slider.min)
        slider.value = np.round(slider.value, 2)

    def mas_clicked(b):
        slider.value = min(slider.value + slider.step, slider.max)
        slider.value = np.round(slider.value, 2)

    menos_btn.on_click(menos_clicked)
    mas_btn.on_click(mas_clicked)

    return slider, widgets.HBox([slider, menos_btn, mas_btn])

def crear_sliders_as(df):
    global slider_widgets_as
    slider_widgets_as = {}
    sliders = []
    incluir = ['ASENTAMIENTO_(CM)']
    num_cols = [col for col in df.columns if col in incluir]

    for col in num_cols:
        col_numeric = pd.to_numeric(df[col], errors='coerce')
        col_min = col_numeric.min(skipna=True) if not col_numeric.dropna().empty else 0.0
        col_max = col_numeric.max(skipna=True) if not col_numeric.dropna().empty else col_min + 1.0
        col_range = col_max - col_min if col_max > col_min else 1.0
        slider, hbox = crear_slider_con_botones_as(col, col_min, col_max, col_range)
        slider_widgets_as[col] = slider
        sliders.append(hbox)

    sliders_box_as.children = sliders

# === Cargar DataFrame ===
def obtener_df_actual_as():
    origen = origen_dropdown_as.value
    df = (datos_csv.get("asentamiento", pd.DataFrame()) if origen == "CSV" else datos_xlsx.get("asentamiento", pd.DataFrame())).copy()
    if not df.empty and "ASENTAMIENTO_(CM)" in df.columns:
        df["ASENTAMIENTO_(CM)"] = pd.to_numeric(df["ASENTAMIENTO_(CM)"], errors='coerce').abs().round(2)
    return df

# === Actualizar selectores dependientes ===
def actualizar_celdas_as(change=None):
    df = obtener_df_actual_as()
    if df.empty:
        celda_dropdown_as.options = []
        return
    prog = progresiva_dropdown_as.value
    if prog == "Todas":
        celdas = sorted(df['CELDA_DE_ASENTAMIENTO'].dropna().unique())
    else:
        celdas = sorted(df[df['PROGRESIVA'] == prog]['CELDA_DE_ASENTAMIENTO'].dropna().unique())
    celda_dropdown_as.options = ["Todas"] + list(celdas)
    celda_dropdown_as.value = "Todas"

def actualizar_opciones_as(change=None):
    df = obtener_df_actual_as()
    if df.empty:
        progresiva_dropdown_as.options = []
        celda_dropdown_as.options = []
        anio_as_dropdown.options = []
        sliders_box_as.children = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresivas = sorted(df['PROGRESIVA'].dropna().unique())
    progresiva_dropdown_as.options = ["Todas"] + list(progresivas)
    progresiva_dropdown_as.value = "Todas"
    actualizar_celdas_as()
    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_as_dropdown.options = ["Todos"] + [str(a) for a in anios]
    anio_as_dropdown.value = "Todos"
    crear_sliders_as(df)

progresiva_dropdown_as.observe(actualizar_celdas_as, names='value')
origen_dropdown_as.observe(actualizar_opciones_as, names='value')
actualizar_opciones_as()

# === Alarmas ===
tipos_alarma_as = [("Todos", "todos"), ("Normal 🟢", "normal"), ("Alerta 🟡", "alerta"), ("Peligro 🔴", "peligro"), ("Sin dato ⚪️", "sin dato")]
alarma_selector_as = widgets.Dropdown(
    options=tipos_alarma_as, value="todos",
    description="Ver alarmas:", layout=widgets.Layout(width='160px')
)

boton_estado_as = widgets.Button(description="Calcular estado", button_style="warning", layout=widgets.Layout(width='150px'))
boton_descargar_as = widgets.Button(description="📥 Descargar XLSX", button_style="success", layout=widgets.Layout(width='180px'))

output_estado_as = widgets.Output()
output_tabla_as = widgets.Output()
output_piechart_as = widgets.Output()
output_descarga_as = widgets.Output()

def clasificar_estado_as(row, umbral):
    valor = pd.to_numeric(row.get('ASENTAMIENTO_(CM)', np.nan), errors='coerce')
    if pd.isna(valor):
        return "sin dato"
    if umbral == 0:
        return "normal"
    if valor > umbral:
        return "peligro"
    if valor > 0.7 * umbral:
        return "alerta"
    return "normal"

def emoji_alarma_as(estado):
    return {"normal": "🟢", "alerta": "🟡", "peligro": "🔴"}.get(estado, "⚪️")

def mostrar_tabla_as(df, tipo="todos", n_filas=20):
    df = df.copy()
    df["alarma"] = df["estado"].map(emoji_alarma_as)
    if tipo != "todos":
        df = df[df["estado"] == tipo]
    mostrar = df if n_filas == 'Todos' else df.head(int(n_filas))
    display(HTML("<b>Tabla de alarmas - Celdas de Asentamiento:</b>"))
    columnas = ["FECHA", "CELDA_DE_ASENTAMIENTO", "PROGRESIVA",
                "ASENTAMIENTO_(CM)", "estado", "alarma"]
    columnas = [col for col in columnas if col in mostrar.columns]
    display(HTML(mostrar[columnas].to_html(index=False, float_format="%.2f")))


df_as_alarma = pd.DataFrame()

def actualizar_tabla_as(change=None):
    with output_tabla_as:
        clear_output(wait=True)
        if not df_as_alarma.empty:
            mostrar_tabla_as(df_as_alarma, tipo=alarma_selector_as.value, n_filas=filas_dropdown_as.value)
        else:
            display(HTML("<b style='color:red'>No hay datos para mostrar.</b>"))

alarma_selector_as.observe(actualizar_tabla_as, names='value')
filas_dropdown_as.observe(actualizar_tabla_as, names='value')

def actualizar_piechart_as():
    with output_piechart_as:
        clear_output(wait=True)
        if not df_as_alarma.empty:
            estado_counts = df_as_alarma['estado'].value_counts()
            colores_map = {"normal": "#8BC34A", "alerta": "#FFC107", "peligro": "#F44336", "sin dato": "#BDBDBD"}
            fig = go.Figure(data=[go.Pie(
                labels=[f"{e} {emoji_alarma_as(e)}" for e in estado_counts.index],
                values=estado_counts.values,
                hole=.5,
                marker_colors=[colores_map.get(e, "#BDBDBD") for e in estado_counts.index]
            )])
            fig.update_layout(title_text="Distribución de estados - Celdas de Asentamiento")
            fig.show()
        else:
            display(HTML("<b style='color:red'>No hay datos para graficar.</b>"))

def calcular_estado_as(b=None):
    with output_estado_as:
        clear_output(wait=True)
        df = obtener_df_actual_as()
        if df.empty:
            display(HTML("<b style='color:red'>No hay datos para esta selección.</b>"))
            return
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        if anio_as_dropdown.value != "Todos":
            try:
                anio_sel = int(anio_as_dropdown.value)
                df = df[df['FECHA'].dt.year == anio_sel]
            except:
                pass
        if progresiva_dropdown_as.value != "Todas":
            df = df[df['PROGRESIVA'] == progresiva_dropdown_as.value]
        if celda_dropdown_as.value != "Todas":
            df = df[df['CELDA_DE_ASENTAMIENTO'] == celda_dropdown_as.value]
        if df.empty:
            display(HTML("<b style='color:red'>No hay datos para esta selección.</b>"))
            return
        umbral = slider_widgets_as['ASENTAMIENTO_(CM)'].value if 'ASENTAMIENTO_(CM)' in slider_widgets_as else 0
        df['estado'] = df.apply(lambda row: clasificar_estado_as(row, umbral), axis=1)
        global df_as_alarma
        df_as_alarma = df.copy()
        actualizar_tabla_as()
        actualizar_piechart_as()

def descargar_as_xlsx(b=None):
    with output_descarga_as:
        clear_output(wait=True)
        if not df_as_alarma.empty:
            fecha_str = datetime.now().strftime("%d-%m-%y")
            nombre_archivo = f"alarmas_celdas_asentamiento_{fecha_str}.xlsx"
            buffer = io.BytesIO()
            columnas_exportar = ["FECHA", "CELDA_DE_ASENTAMIENTO", "PROGRESIVA",
                                 "ASENTAMIENTO_(CM)", "estado"]
            columnas = [col for col in columnas_exportar if col in df_as_alarma.columns]
            df_as_alarma[columnas].to_excel(buffer, index=False)
            buffer.seek(0)
            b64 = base64.b64encode(buffer.read()).decode()
            href = f'<a download="{nombre_archivo}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}">📥 Descargar {nombre_archivo}</a>'
            display(HTML(href))
        else:
            display(HTML("<b style='color:red'>No hay datos para descargar.</b>"))

boton_estado_as.on_click(calcular_estado_as)
boton_descargar_as.on_click(descargar_as_xlsx)

# === Display final ===
as_controles = widgets.VBox([
    widgets.HBox([origen_dropdown_as, progresiva_dropdown_as, celda_dropdown_as, anio_as_dropdown]),
    widgets.HTML("<b>Ajustá los umbrales para clasificar alarmas:</b>"),
    sliders_box_as,
    widgets.HBox([filas_dropdown_as, alarma_selector_as, boton_estado_as, boton_descargar_as]),
    output_descarga_as
])

display(widgets.HTML("<h3 style='color:#1866a3'>Estado/alarma automático - Celdas de Asentamiento</h3>"))
display(as_controles, output_estado_as, output_tabla_as, output_piechart_as)


HTML(value="<h3 style='color:#1866a3'>Estado/alarma automático - Celdas de Asentamiento</h3>")

Output()

Output()

Output()

#Script Estado/Alerta Freatímetros

In [12]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io
from datetime import datetime
import base64

# === Selectores ===
origen_dropdown_fr = widgets.Dropdown(
    options=["CSV", "XLSX"],
    value="CSV",
    description="Origen:",
    layout=widgets.Layout(width='160px')
)
freatimetro_dropdown_fr = widgets.Dropdown(description="Freatímetro:", layout=widgets.Layout(width='180px'))
anio_fr_dropdown = widgets.Dropdown(description="Año:", layout=widgets.Layout(width='100px'))
filas_dropdown_fr = widgets.Dropdown(
    options=[10, 20, 50, 100, 500, 1000, 'Todos'],
    value=20,
    description='Filas:',
    layout=widgets.Layout(width='100px')
)

slider_widgets_fr = {}
sliders_box_fr = widgets.VBox()

def crear_slider_con_botones_fr(col, col_min, col_max, col_range):
    slider = widgets.FloatSlider(
        value=0.0,
        min=col_min,
        max=col_max,
        step=0.1,
        description=f"{col}:",
        continuous_update=False,
        readout=True,
        readout_format='.2f',
        layout=widgets.Layout(width='360px'),
        style={'description_width': '180px'}
    )
    menos_btn = widgets.Button(description='-', layout=widgets.Layout(width='32px'))
    mas_btn = widgets.Button(description='+', layout=widgets.Layout(width='32px'))

    def menos_clicked(b):
        slider.value = max(slider.value - slider.step, slider.min)
        slider.value = np.round(slider.value, 2)

    def mas_clicked(b):
        slider.value = min(slider.value + slider.step, slider.max)
        slider.value = np.round(slider.value, 2)

    menos_btn.on_click(menos_clicked)
    mas_btn.on_click(mas_clicked)

    return slider, widgets.HBox([slider, menos_btn, mas_btn])

def crear_sliders_fr(df):
    global slider_widgets_fr
    slider_widgets_fr = {}
    sliders = []
    incluir = ['CARGA_(M.C.A)']
    for col in incluir:
        col_numeric = pd.to_numeric(df[col], errors='coerce')
        col_min, col_max = 0, 20  # Ajuste del rango del slider de carga
        col_range = col_max - col_min
        slider, hbox = crear_slider_con_botones_fr(col, col_min, col_max, col_range)
        slider_widgets_fr[col] = slider
        sliders.append(hbox)
    sliders_box_fr.children = sliders

def obtener_df_actual_fr():
    origen = origen_dropdown_fr.value
    return (datos_csv.get("freatimetros", pd.DataFrame()) if origen == "CSV" else datos_xlsx.get("freatimetros", pd.DataFrame())).copy()

def actualizar_opciones_fr(change=None):
    df = obtener_df_actual_fr()
    if df.empty:
        freatimetro_dropdown_fr.options = []
        anio_fr_dropdown.options = []
        sliders_box_fr.children = []
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    freatimetros = sorted(df['FREATIMETRO'].dropna().unique())
    valor_actual = freatimetro_dropdown_fr.value
    freatimetro_dropdown_fr.options = ["Todos"] + list(freatimetros)
    if valor_actual in freatimetro_dropdown_fr.options:
        freatimetro_dropdown_fr.value = valor_actual
    else:
        freatimetro_dropdown_fr.value = "Todos"
    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_fr_dropdown.options = ["Todos"] + [str(a) for a in anios]
    anio_fr_dropdown.value = "Todos"
    crear_sliders_fr(df)

def clasificar_estado_fr(row, umbrales):
    for var in umbrales:
        valor = pd.to_numeric(row.get(var, np.nan), errors='coerce')
        if pd.isna(valor):
            return "sin dato"
    if all(umbrales[var] == 0 for var in umbrales):
        return "normal"
    if any(pd.to_numeric(row[var], errors='coerce') > umbrales[var] for var in umbrales if umbrales[var] != 0):
        return "peligro"
    if any(pd.to_numeric(row[var], errors='coerce') > 0.7 * umbrales[var] for var in umbrales if umbrales[var] != 0):
        return "alerta"
    return "normal"

tipos_alarma_fr = [("Todos", "todos"), ("Normal 🟢", "normal"), ("Alerta 🟡", "alerta"), ("Peligro 🔴", "peligro"), ("Sin dato ⚪️", "sin dato")]
alarma_selector_fr = widgets.Dropdown(
    options=tipos_alarma_fr, value="todos",
    description="Ver alarmas:", layout=widgets.Layout(width='140px')
)

boton_estado_fr = widgets.Button(description="Calcular estado", button_style="warning", layout=widgets.Layout(width='140px'))
boton_descargar_fr = widgets.Button(description="📥 Descargar XLSX", button_style="success", layout=widgets.Layout(width='180px'))

output_estado_fr = widgets.Output()
output_tabla_fr = widgets.Output()
output_piechart_fr = widgets.Output()
output_descarga_fr = widgets.Output()

df_fr_alarma = pd.DataFrame()

def calcular_estado_fr(b=None):
    with output_estado_fr:
        clear_output(wait=True)
        df = obtener_df_actual_fr()
        if df.empty:
            display(HTML("<b style='color:red'>No hay datos para esta selección.</b>"))
            return
        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
        if anio_fr_dropdown.value != "Todos":
            try:
                anio_sel = int(anio_fr_dropdown.value)
                df = df[df['FECHA'].dt.year == anio_sel]
            except:
                pass
        if freatimetro_dropdown_fr.value != "Todos":
            df = df[df['FREATIMETRO'] == freatimetro_dropdown_fr.value]
        if df.empty:
            display(HTML("<b style='color:red'>No hay datos para esta selección.</b>"))
            return
        umbrales = {var: slider.value for var, slider in slider_widgets_fr.items()}
        df['estado'] = df.apply(lambda row: clasificar_estado_fr(row, umbrales), axis=1)
        global df_fr_alarma
        df_fr_alarma = df.copy()
        actualizar_tabla_fr()
        actualizar_piechart_fr()

def mostrar_tabla_fr(df, tipo="todos", n_filas=20):
    df = df.copy()
    df["alarma"] = df["estado"].map(lambda e: {"normal": "🟢", "alerta": "🟡", "peligro": "🔴"}.get(e, "⚪️"))
    if tipo != "todos":
        df = df[df["estado"] == tipo]
    mostrar = df if n_filas == 'Todos' else df.head(int(n_filas))
    display(HTML("<b>Tabla de alarmas - Freatímetros:</b>"))
    columnas = ["FECHA", "FREATIMETRO", "CARGA_(M.C.A)", "estado", "alarma"]
    columnas = [col for col in columnas if col in mostrar.columns]
    display(HTML(mostrar[columnas].to_html(index=False)))

def actualizar_tabla_fr(change=None):
    with output_tabla_fr:
        clear_output(wait=True)
        if not df_fr_alarma.empty:
            mostrar_tabla_fr(df_fr_alarma, tipo=alarma_selector_fr.value, n_filas=filas_dropdown_fr.value)
        else:
            display(HTML("<b style='color:red'>No hay datos para mostrar.</b>"))

def actualizar_piechart_fr():
    with output_piechart_fr:
        clear_output(wait=True)
        if not df_fr_alarma.empty:
            estado_counts = df_fr_alarma['estado'].value_counts()
            colores_map = {"normal": "#8BC34A", "alerta": "#FFC107", "peligro": "#F44336", "sin dato": "#BDBDBD"}
            fig = go.Figure(data=[go.Pie(
                labels=[f"{e} " for e in estado_counts.index],
                values=estado_counts.values,
                hole=.5,
                marker_colors=[colores_map.get(e, "#BDBDBD") for e in estado_counts.index]
            )])
            fig.update_layout(title_text="Distribución de estados - Freatímetros")
            fig.show()
        else:
            display(HTML("<b style='color:red'>No hay datos para graficar.</b>"))

def descargar_fr_xlsx(b=None):
    with output_descarga_fr:
        clear_output(wait=True)
        if not df_fr_alarma.empty:
            fecha_str = datetime.now().strftime("%d-%m-%y")
            nombre_archivo = f"alarmas_freatimetros_{fecha_str}.xlsx"
            buffer = io.BytesIO()
            columnas_exportar = ["FECHA", "FREATIMETRO", "CARGA_(M.C.A)", "estado"]
            columnas = [col for col in columnas_exportar if col in df_fr_alarma.columns]
            df_fr_alarma[columnas].to_excel(buffer, index=False)
            buffer.seek(0)
            b64 = base64.b64encode(buffer.read()).decode()
            href = f'<a download="{nombre_archivo}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}">📥 Descargar {nombre_archivo}</a>'
            display(HTML(href))
        else:
            display(HTML("<b style='color:red'>No hay datos para descargar.</b>"))

boton_estado_fr.on_click(calcular_estado_fr)
boton_descargar_fr.on_click(descargar_fr_xlsx)

freatimetro_dropdown_fr.observe(actualizar_opciones_fr, names='value')
origen_dropdown_fr.observe(actualizar_opciones_fr, names='value')
actualizar_opciones_fr()

fr_controles = widgets.VBox([
    widgets.HBox([origen_dropdown_fr, freatimetro_dropdown_fr, anio_fr_dropdown]),
    widgets.HTML("<b>Ajustá los umbrales para clasificar alarmas:</b>"),
    sliders_box_fr,
    widgets.HBox([filas_dropdown_fr, alarma_selector_fr, boton_estado_fr, boton_descargar_fr]),
    output_descarga_fr
])

display(widgets.HTML("<h3 style='color:#1866a3'>Estado/alarma automático - Freatímetros</h3>"))
display(fr_controles, output_estado_fr, output_tabla_fr, output_piechart_fr)


HTML(value="<h3 style='color:#1866a3'>Estado/alarma automático - Freatímetros</h3>")

Output()

Output()

Output()

#Script Estado/Alerta Extensómetros

In [13]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import io
from datetime import datetime
import base64

# === Selectores ===
origen_dropdown_ext = widgets.Dropdown(
    options=["CSV", "XLSX"],
    value="CSV",
    description="Origen:",
    layout=widgets.Layout(width='160px')
)
extensometro_dropdown_ext = widgets.Dropdown(description="Extensómetro:", layout=widgets.Layout(width='180px'))
anio_ext_dropdown = widgets.Dropdown(description="Año:", layout=widgets.Layout(width='100px'))
filas_dropdown_ext = widgets.Dropdown(
    options=[10, 20, 50, 100, 500, 1000, 'Todos'],
    value=20,
    description='Filas:',
    layout=widgets.Layout(width='100px')
)

slider_widgets_ext = {}
sliders_box_ext = widgets.VBox()

def crear_slider_con_botones_ext(col, col_min, col_max, col_range):
    slider = widgets.FloatSlider(
        value=0.0,
        min=col_min,
        max=col_max,
        step=0.1,
        description=f"{col}:",
        continuous_update=False,
        readout=True,
        readout_format='.2f',
        layout=widgets.Layout(width='360px'),
        style={'description_width': '180px'}
    )
    menos_btn = widgets.Button(description='-', layout=widgets.Layout(width='32px'))
    mas_btn = widgets.Button(description='+', layout=widgets.Layout(width='32px'))

    def menos_clicked(b):
        slider.value = max(slider.value - slider.step, slider.min)
        slider.value = np.round(slider.value, 2)

    def mas_clicked(b):
        slider.value = min(slider.value + slider.step, slider.max)
        slider.value = np.round(slider.value, 2)

    menos_btn.on_click(menos_clicked)
    mas_btn.on_click(mas_clicked)

    return slider, widgets.HBox([slider, menos_btn, mas_btn])

def crear_sliders_ext(df):
    global slider_widgets_ext
    slider_widgets_ext = {}
    sliders = []

    if df.empty:
        sliders_box_ext.children = []
        return

    # Buscar únicamente la columna "ACUMULADO"
    columna_acumulado = None
    for col in df.columns:
        if "ACUMULADO" in col.upper():
            columna_acumulado = col
            break

    if columna_acumulado:
        print(f"Columna acumulado detectada: {columna_acumulado}")

        col_numeric = pd.to_numeric(df[columna_acumulado], errors='coerce')
        col_min, col_max = 0, 30  # Rango por defecto para extensómetros

        # Intentar usar valores reales para un mejor rango
        valores_validos = col_numeric.dropna()
        if len(valores_validos) > 0:
            col_max = max(50, valores_validos.max() * 1.2)  # 20% más que el máximo

        slider, hbox = crear_slider_con_botones_ext(columna_acumulado, col_min, col_max, col_max - col_min)
        slider_widgets_ext[columna_acumulado] = slider
        sliders.append(hbox)
    else:
        print("No se encontró columna 'ACUMULADO' en los datos")

    sliders_box_ext.children = sliders

def obtener_df_actual_ext():
    origen = origen_dropdown_ext.value
    return (datos_csv.get("extensometro", pd.DataFrame()) if origen == "CSV" else datos_xlsx.get("extensometro", pd.DataFrame())).copy()

def actualizar_opciones_ext(change=None):
    df = obtener_df_actual_ext()
    if df.empty:
        extensometro_dropdown_ext.options = []
        anio_ext_dropdown.options = []
        sliders_box_ext.children = []
        return

    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')

    # Detectar columna de identificación del extensómetro
    col_extensometro = None
    posibles_cols = ['EXTENSOMETRO', 'EXTENSÓMETRO', 'EXT', 'INSTRUMENTO', 'ID']
    for col in posibles_cols:
        if col in df.columns:
            col_extensometro = col
            break

    if col_extensometro:
        extensometros = sorted(df[col_extensometro].dropna().unique())
        valor_actual = extensometro_dropdown_ext.value
        extensometro_dropdown_ext.options = ["Todos"] + list(extensometros)
        if valor_actual in extensometro_dropdown_ext.options:
            extensometro_dropdown_ext.value = valor_actual
        else:
            extensometro_dropdown_ext.value = "Todos"
    else:
        extensometro_dropdown_ext.options = ["Todos"]
        extensometro_dropdown_ext.value = "Todos"

    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    anio_ext_dropdown.options = ["Todos"] + [str(a) for a in anios]
    anio_ext_dropdown.value = "Todos"
    crear_sliders_ext(df)

def clasificar_estado_ext(row, umbrales):
    for var in umbrales:
        valor = pd.to_numeric(row.get(var, np.nan), errors='coerce')
        if pd.isna(valor):
            return "sin dato"
    if all(umbrales[var] == 0 for var in umbrales):
        return "normal"
    if any(pd.to_numeric(row[var], errors='coerce') > umbrales[var] for var in umbrales if umbrales[var] != 0):
        return "peligro"
    if any(pd.to_numeric(row[var], errors='coerce') > 0.7 * umbrales[var] for var in umbrales if umbrales[var] != 0):
        return "alerta"
    return "normal"

tipos_alarma_ext = [("Todos", "todos"), ("Normal 🟢", "normal"), ("Alerta 🟡", "alerta"), ("Peligro 🔴", "peligro"), ("Sin dato ⚪️", "sin dato")]
alarma_selector_ext = widgets.Dropdown(
    options=tipos_alarma_ext, value="todos",
    description="Ver alarmas:", layout=widgets.Layout(width='150px')
)

boton_estado_ext = widgets.Button(description="Calcular estado", button_style="warning", layout=widgets.Layout(width='140px'))
boton_descargar_ext = widgets.Button(description="📥 Descargar XLSX", button_style="success", layout=widgets.Layout(width='180px'))

output_estado_ext = widgets.Output()
output_tabla_ext = widgets.Output()
output_piechart_ext = widgets.Output()
output_descarga_ext = widgets.Output()

df_ext_alarma = pd.DataFrame()

def calcular_estado_ext(b=None):
    with output_estado_ext:
        clear_output(wait=True)
        df = obtener_df_actual_ext()
        if df.empty:
            display(HTML("<b style='color:red'>No hay datos para esta selección.</b>"))
            return

        df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')

        if anio_ext_dropdown.value != "Todos":
            try:
                anio_sel = int(anio_ext_dropdown.value)
                df = df[df['FECHA'].dt.year == anio_sel]
            except:
                pass

        # Detectar columna de identificación del extensómetro
        col_extensometro = None
        posibles_cols = ['EXTENSOMETRO', 'EXTENSÓMETRO', 'EXT', 'INSTRUMENTO', 'ID']
        for col in posibles_cols:
            if col in df.columns:
                col_extensometro = col
                break

        if extensometro_dropdown_ext.value != "Todos" and col_extensometro:
            df = df[df[col_extensometro] == extensometro_dropdown_ext.value]

        if df.empty:
            display(HTML("<b style='color:red'>No hay datos para esta selección.</b>"))
            return

        umbrales = {var: slider.value for var, slider in slider_widgets_ext.items()}
        df['estado'] = df.apply(lambda row: clasificar_estado_ext(row, umbrales), axis=1)

        global df_ext_alarma
        df_ext_alarma = df.copy()
        actualizar_tabla_ext()
        actualizar_piechart_ext()

def mostrar_tabla_ext(df, tipo="todos", n_filas=20):
    df = df.copy()
    df["alarma"] = df["estado"].map(lambda e: {"normal": "🟢", "alerta": "🟡", "peligro": "🔴"}.get(e, "⚪️"))
    if tipo != "todos":
        df = df[df["estado"] == tipo]
    mostrar = df if n_filas == 'Todos' else df.head(int(n_filas))
    display(HTML("<b>Tabla de alarmas - Extensómetros:</b>"))

    # Detectar columnas disponibles
    col_extensometro = None
    posibles_cols = ['EXTENSOMETRO', 'EXTENSÓMETRO', 'EXT', 'INSTRUMENTO', 'ID']
    for col in posibles_cols:
        if col in mostrar.columns:
            col_extensometro = col
            break

    columnas = ["FECHA"]
    if col_extensometro:
        columnas.append(col_extensometro)

    # Agregar todas las columnas numéricas que tienen slider
    for col in slider_widgets_ext.keys():
        if col in mostrar.columns:
            columnas.append(col)

    columnas.extend(["estado", "alarma"])
    columnas = [col for col in columnas if col in mostrar.columns]
    display(HTML(mostrar[columnas].to_html(index=False)))

def actualizar_tabla_ext(change=None):
    with output_tabla_ext:
        clear_output(wait=True)
        if not df_ext_alarma.empty:
            mostrar_tabla_ext(df_ext_alarma, tipo=alarma_selector_ext.value, n_filas=filas_dropdown_ext.value)
        else:
            display(HTML("<b style='color:red'>No hay datos para mostrar.</b>"))

def actualizar_piechart_ext():
    with output_piechart_ext:
        clear_output(wait=True)
        if not df_ext_alarma.empty:
            estado_counts = df_ext_alarma['estado'].value_counts()
            colores_map = {"normal": "#8BC34A", "alerta": "#FFC107", "peligro": "#F44336", "sin dato": "#BDBDBD"}
            fig = go.Figure(data=[go.Pie(
                labels=[f"{e} " for e in estado_counts.index],
                values=estado_counts.values,
                hole=.5,
                marker_colors=[colores_map.get(e, "#BDBDBD") for e in estado_counts.index]
            )])
            fig.update_layout(title_text="Distribución de estados - Extensómetros")
            fig.show()
        else:
            display(HTML("<b style='color:red'>No hay datos para graficar.</b>"))

def descargar_ext_xlsx(b=None):
    with output_descarga_ext:
        clear_output(wait=True)
        if not df_ext_alarma.empty:
            fecha_str = datetime.now().strftime("%d-%m-%y")
            nombre_archivo = f"alarmas_extensometros_{fecha_str}.xlsx"
            buffer = io.BytesIO()

            # Detectar columnas disponibles
            col_extensometro = None
            posibles_cols = ['EXTENSOMETRO', 'EXTENSÓMETRO', 'EXT', 'INSTRUMENTO', 'ID']
            for col in posibles_cols:
                if col in df_ext_alarma.columns:
                    col_extensometro = col
                    break

            columnas_exportar = ["FECHA"]
            if col_extensometro:
                columnas_exportar.append(col_extensometro)

            # Agregar todas las columnas numéricas que tienen slider
            for col in slider_widgets_ext.keys():
                if col in df_ext_alarma.columns:
                    columnas_exportar.append(col)

            columnas_exportar.append("estado")
            columnas = [col for col in columnas_exportar if col in df_ext_alarma.columns]

            df_ext_alarma[columnas].to_excel(buffer, index=False)
            buffer.seek(0)
            b64 = base64.b64encode(buffer.read()).decode()
            href = f'<a download="{nombre_archivo}" href="data:application/vnd.openxmlformats-officedocument.spreadsheetml.sheet;base64,{b64}">📥 Descargar {nombre_archivo}</a>'
            display(HTML(href))
        else:
            display(HTML("<b style='color:red'>No hay datos para descargar.</b>"))

# Conectar eventos
boton_estado_ext.on_click(calcular_estado_ext)
boton_descargar_ext.on_click(descargar_ext_xlsx)
alarma_selector_ext.observe(actualizar_tabla_ext, names='value')
filas_dropdown_ext.observe(actualizar_tabla_ext, names='value')

extensometro_dropdown_ext.observe(actualizar_opciones_ext, names='value')
origen_dropdown_ext.observe(actualizar_opciones_ext, names='value')
actualizar_opciones_ext()

# Interfaz
ext_controles = widgets.VBox([
    widgets.HBox([origen_dropdown_ext, extensometro_dropdown_ext, anio_ext_dropdown]),
    widgets.HTML("<b>Ajustá los umbrales para clasificar alarmas:</b>"),
    sliders_box_ext,
    widgets.HBox([filas_dropdown_ext, alarma_selector_ext, boton_estado_ext, boton_descargar_ext]),
    output_descarga_ext
])

display(widgets.HTML("<h3 style='color:#1866a3'>Estado/alarma automático - Extensómetros</h3>"))
display(ext_controles, output_estado_ext, output_tabla_ext, output_piechart_ext)

Columna acumulado detectada: ACUMULADO_(MM)
Columna acumulado detectada: ACUMULADO_(MM)


HTML(value="<h3 style='color:#1866a3'>Estado/alarma automático - Extensómetros</h3>")

Output()

Output()

Output()